In [ ]:
!pip install transformers
!pip install pytorch
!pip install timm
!pip install --upgrade transformers
!pip install transformers==<desired_version>
!pip install flask


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pytorch
  Running setup.py clean for pytorch
Failed to build pytorch
ERROR: Could not build wheels for pytorch, which is required to install pyproject.toml-based projects
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 12.1 MB/s eta 0:00:00
/bin/bash: -c: line 1: syntax error near unexpected token `newline'
/bin/bash: -c: line 1: `pip install transformers==<des

In [2]:
from transformers import DetrImageProcessor, DetrForObjectDetection
import torch
from PIL import Image
import ipywidgets as widgets
from IPython.display import display, clear_output
import io

# Create the file uploader widget
uploader = widgets.FileUpload(description="Upload Image", multiple=False)
display(uploader)

# Create a loading button
loading_button = widgets.Button(description="Detecting...", disabled=True)

# Create a button to clear the uploaded image
clear_button = widgets.Button(description="Clear Image", disabled=True)

# Define the function to clear the uploaded image
def clear_uploaded_image(_):
    uploader.value.clear()
    clear_output()
    print("Image cleared.")
    loading_button.disabled = True
    clear_button.disabled = True

clear_button.on_click(clear_uploaded_image)
display(clear_button)

# Define the function to process the uploaded image
def process_uploaded_image(change):
    with output:
        clear_output()  # Clear previous outputs

        if len(uploader.data) == 0:
            print("Please upload an image.")
            loading_button.disabled = True
            clear_button.disabled = True
            return

        uploaded_image = Image.open(io.BytesIO(uploader.data[-1]))
        display(loading_button)
        clear_button.disabled = False

        # Your existing code for object detection
        processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")
        model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50")

        # Calculate the size for object detection using the longest edge
        max_size = max(uploaded_image.size)
        target_sizes = torch.tensor([[max_size, max_size]])

        inputs = processor(images=uploaded_image, return_tensors="pt", target_sizes=target_sizes)
        outputs = model(**inputs)

        results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.9)[0]

        if len(results["scores"]) == 0:
            print("I am not able to recognize the object.")
        else:
            for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
                box = [round(i, 2) for i in box.tolist()]
                print(
                    f"Detected {model.config.id2label[label.item()]} with confidence "
                    f"{round(score.item(), 3)} at location {box}"
                )

        clear_output(wait=True)  # Clear previous outputs
        clear_button.disabled = False  # Enable the clear button
        loading_button.disabled = True  # Disable the detecting button

# Attach the observer to the uploader widget
uploader.observe(process_uploaded_image, names='data')

# Create an output area to display results
output = widgets.Output()
display(output)

FileUpload(value={}, description='Upload Image')

Button(description='Clear Image', disabled=True, style=ButtonStyle())

Output()